A neural network consist of 2 cnn layers and 4 fully connected layers.

Source: https://github.com/jojonki/cnn-for-sentence-classification





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [ ]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout
from keras.layers import Conv2D, MaxPooling1D,Conv1D
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0,converters={'body': eval})
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [ ]:
len(labels)

78

In [ ]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 300


In [ ]:
freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

زبان


In [ ]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [ ]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 225345
129280


In [ ]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# print(vecX[0])

train:  (12935, 300) 
test:  (4312, 300) 
val:  (4312, 300) 
y_tain: (12935, 78)


In [ ]:
embd_dim = 300


# ***If the word2vec model is not generated before, we should run the next block.***

In [ ]:
# embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
# embed_model.save('word2vec_model')

# ***Otherwise, we can run the next block.***

In [ ]:
embed_model=gensim.models.Word2Vec.load('word2vec_model')

In [ ]:
word2vec_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  if word in embed_model.wv.vocab:
        embedding_vector =embed_model[word]
  
        # words not found in embedding index will be all-zeros.
        word2vec_embd_w[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [ ]:
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[word2vec_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    embd_sentence = Permute((2,1))(embd_sentence)
    embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # cnn
    cnn = Conv2D(1, 
                 kernel_size=(5, sentence_maxlen),
                 activation='relu')(embd_sentence)
    print(cnn.shape)
    cnn =  Lambda(lambda x: K.sum(x, axis=3))(cnn)
    print(cnn.shape)
    cnn = MaxPooling1D(3)(cnn)
    print(cnn.shape)
    

    cnn1 = Conv1D(1, 
                 kernel_size=(3),
                 activation='relu')(cnn)
    print(cnn1.shape)
    # cnn1 =  Lambda(lambda x: K.sum(x, axis=3))(cnn1)
    print(cnn1.shape)
    cnn1 = MaxPooling1D(3)(cnn1)
    print(cnn1.shape)

    cnn1 = Lambda(lambda x: K.sum(x, axis=2))(cnn1)
    print(cnn1.shape)
    
    hidden1=Dense(400,activation="relu")(cnn1)
    hidden2=Dense(300,activation="relu")(hidden1)
    hidden3=Dense(200,activation="relu")(hidden2)
    hidden4=Dense(150,activation="relu")(hidden3)
    out = Dense(len(labels), activation='sigmoid')(hidden4)
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer=sgd, loss='binary_crossentropy',metrics=["accuracy","categorical_accuracy"]) 

    return model

model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)
print(model.summary())


(None, 296, 1, 1)
(None, 296, 1)
(None, 98, 1)
(None, 96, 1)
(None, 96, 1)
(None, 32, 1)
(None, 32)
Model: "sentence_claccification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   [(None, 300)]             0         
_________________________________________________________________
shared_embd (Embedding)      (None, 300, 300)          67603500  
_________________________________________________________________
permute_13 (Permute)         (None, 300, 300)          0         
_________________________________________________________________
lambda_26 (Lambda)           (None, 300, 300, 1)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 296, 1, 1)         1501      
_________________________________________________________________
lambda_27 (Lambda)           (None, 296, 1)            0         
_________

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) # Model stop training after 5 epoch where validation loss didnt decrease
mc = ModelCheckpoint('best_2cnn_4fc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True) #You save model weight at the epoch where validation loss is minimal
model.fit(X_train, y_train, batch_size=32,epochs=250,verbose=1,validation_data=(X_val, y_val),callbacks=[es,mc])#you can run for 1000 epoch btw model will stop after 50 epoch without better validation loss

Epoch 1/250
405/405 [==============================] - 52s 128ms/step - loss: 0.1096 - accuracy: 0.0247 - categorical_accuracy: 0.0247 - val_loss: 0.1103 - val_accuracy: 0.0313 - val_categorical_accuracy: 0.0313

Epoch 00001: val_loss improved from inf to 0.11033, saving model to best_2cnn_4fc.h5
Epoch 2/250
405/405 [==============================] - 52s 130ms/step - loss: 0.1084 - accuracy: 0.0239 - categorical_accuracy: 0.0239 - val_loss: 0.1097 - val_accuracy: 0.0174 - val_categorical_accuracy: 0.0174

Epoch 00002: val_loss improved from 0.11033 to 0.10968, saving model to best_2cnn_4fc.h5
Epoch 3/250
405/405 [==============================] - 52s 129ms/step - loss: 0.1078 - accuracy: 0.0268 - categorical_accuracy: 0.0268 - val_loss: 0.1092 - val_accuracy: 0.0144 - val_categorical_accuracy: 0.0144

Epoch 00003: val_loss improved from 0.10968 to 0.10917, saving model to best_2cnn_4fc.h5
Epoch 4/250
405/405 [==============================] - 52s 129ms/step - loss: 0.1073 - accuracy: 0

# ***If the model is generated before:***

In [ ]:
model = load_model('best_2cnn_4fc.h5')
# model.save('CNN_1_no_binary.h5')

In [ ]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [ ]:
print(pred[0])#example

[0.02024955 0.00667295 0.00178918 0.00236279 0.00247431 0.00691393
 0.01946878 0.00634056 0.03716832 0.0354456  0.01223555 0.00819591
 0.00087956 0.04503295 0.02509329 0.02411515 0.00897041 0.00534847
 0.00477275 0.00618264 0.22035608 0.04265302 0.02280614 0.01406318
 0.00928885 0.00439137 0.01184359 0.02140903 0.01753116 0.00310951
 0.00151864 0.02305993 0.00380963 0.00953022 0.00869453 0.02099335
 0.0071758  0.00427705 0.00119334 0.02258196 0.00046957 0.00484139
 0.00782707 0.01723528 0.00064176 0.00388497 0.00216827 0.00507525
 0.00366697 0.0005756  0.00312608 0.00730112 0.00128165 0.01949522
 0.30469364 0.00419578 0.00253981 0.00418487 0.01426223 0.05932781
 0.00438216 0.06035104 0.01874197 0.00451577 0.12233314 0.03575709
 0.00395685 0.15806323 0.01084283 0.01388755 0.01469615 0.00656965
 0.01421803 0.04189262 0.04368302 0.01519099 0.00621146 0.00289363]


In [ ]:
y_pred=[]
measure = np.mean(pred[0]) + 1.15*np.sqrt(np.var(pred[0]))
for l in pred:
  temp=[]
  for value in l:
    if value >= measure:
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [ ]:
measure

0.07626031823456286

In [ ]:
from sklearn.metrics import classification_report,accuracy_score

print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy= 0.07050092764378478
              precision    recall  f1-score   support

           0       0.13      0.68      0.21       106
           1       0.16      0.71      0.27       143
           2       0.19      0.86      0.31       138
           3       0.12      0.21      0.16        14
           4       0.65      0.91      0.76        79
           5       0.00      0.00      0.00        23
           6       0.18      0.81      0.30       142
           7       0.22      0.37      0.28        35
           8       0.10      0.16      0.12        25
           9       0.11      0.42      0.17       109
          10       0.16      0.83      0.27       138
          11       0.70      0.98      0.81        85
          12       0.89      0.95      0.92       131
          13       0.04      0.20      0.07        10
          14       0.13      0.27      0.18        22
          15       0.12      0.66      0.21       137
          16       0.26      0.74      0.38       1

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report,accuracy_score
print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy= 0.21335807050092764
              precision    recall  f1-score   support

           0       0.30      0.40      0.34       123
           1       0.16      0.39      0.23       132
           2       0.39      0.57      0.46       149
           3       0.30      0.32      0.31        19
           4       0.96      0.93      0.94        82
           5       0.00      0.00      0.00        11
           6       0.30      0.59      0.40       152
           7       0.54      0.15      0.23        48
           8       0.18      0.15      0.16        34
           9       0.17      0.15      0.16       123
          10       0.32      0.43      0.37       161
          11       0.85      0.97      0.91        72
          12       0.90      0.97      0.94       142
          13       0.43      0.20      0.27        15
          14       0.00      0.00      0.00        18
          15       0.21      0.47      0.29       148
          16       0.38      0.66      0.48       1

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
